In [7]:
from torch.utils.data import Dataset
import numpy as np
from tqdm import tqdm
from PIL import Image
# from openTSNE import TSNE
import matplotlib.pyplot as plt
import torch

import copy
# import pandas as pd
# import seaborn as sns
import random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch
import torch.nn as nn
import torchvision
import torchvision.utils as utils
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import CDCGAN_size32

from classify_model.resnet import ResNet18

class Mydata_sets2(Dataset):

    def __init__(self, zs,labels):
        super(Mydata_sets2, self).__init__()
        self.zs = zs
        self.labels = labels

    def __getitem__(self, index):
        z = self.zs[index]
        labels = self.labels[index]
        return z, labels

    def __len__(self):
        return len(self.zs)
# ResNet20 = model_all.get_DNN_model(dataset_type, model_name)
# ResNet20.load_state_dict( torch.load("./model_files/CIFAR10/checkpoints/classify_model/ResNet20.pt", map_location=device)) # 这个pt文件里不仅仅是参数，包括了模型
# # ResNet50 = torch.load("./model_files/CIFAR10/checkpoints/classify_model/ResNet50.pt", map_location=device)
# ResNet20 = ResNet20.to(device)
# ResNet20.eval()
# 加载整个 state_dict
checkpoint = torch.load('classify_model/gtsrb_ResNet18_E87_97.85.pth', map_location=device)

# 提取模型参数
state_dict = checkpoint['model']
model = ResNet18().to(device)
model.load_state_dict(state_dict)
model.eval()

feature_model = copy.deepcopy(model)
feature_model.fc = nn.Identity() # 相当于取消fc层, 这样


# checkpoints_g = "./model_files/CIFAR10/checkpoints/BigGAN/model=G-best-weights-step=392000.pth"  # kjl测试#############
# global G
# print("数据类型为：", dataset_type)
# G = model_all.get_generative_model("CIFAR10").to(device)
# G.load_state_dict(torch.load(checkpoints_path, map_location=device)["state_dict"])
# G.eval()
G = CDCGAN_size32.generator(128).to(device)
G.load_state_dict(torch.load('GTSRB_cDCGAN_results/GTSRB_cDCGAN_generator_param_size32_epoch20.pth'))
G.eval()
save_path = 'random_size32_50k/'


data_z_path = 'GTSRB_cDCGAN_results/cGAN_100z_size32_50k.pt'
label_path = 'GTSRB_cDCGAN_results/cGAN_label_size32_50k.pt'
dict_zs = torch.load(data_z_path, map_location="cpu")  # 因为我之前保存数据到了GPU上，所以要回到cpu上才不会出错
data_z_labels = torch.load(label_path,map_location="cpu")

zs_datasets = Mydata_sets2(dict_zs,data_z_labels)
    # 不需要前向传播，一次可以多处理一些
zs_loader = DataLoader(zs_datasets, batch_size=32, shuffle=False, num_workers=0)  # 指定读取配置信息


features, labels, max_values, ids, ten_Ds, picture = [], [], [], [], [], [] # features：提取的2048维图片特征，labels：模型预测的标签，ids图片文件编号
# 每个类别的onehot
onehot = torch.zeros(43, 43)
onehot = onehot.scatter_(1, torch.LongTensor(list(range(43))).view(43, 1), 1).view(43,43, 1, 1)


with torch.no_grad():  # 取消梯度计算，加快运行速度
    for batch_z, batch_labels in zs_loader:
        # z = torch.tensor(batch_z, dtype=torch.float32, device=device).detach()
        label_onehot = onehot[batch_labels]
        batch_z = batch_z.to(device)
        # z = torch.tensor(batch_z).to(torch.float32).to(device).detach()  # latent code
        imgs = G(batch_z,label_onehot.to(device))

        feature = feature_model(imgs)  # N, 2048
        features.append(feature)
        # DNN_model.layer3.register_forward_hook(get_activation('layer3'))
        layers = model(imgs)  # 分类模型分类图片
        # CAMlayer = activation['layer3']
        max_value, label = torch.max(layers, dim=1)
        same_labels_mask = torch.eq(label, batch_labels.to(device))
        # 将不同标签对应的置信度设为负值
        max_value = torch.where(same_labels_mask, max_value, -max_value)
        picture.append(imgs)
        # label = torch.argmax(ten_D, dim=1)
        ten_Ds.append(layers)
        labels.append(label)
        max_values.append(max_value)

picture = torch.cat(picture, dim=0).squeeze().view(50000,-1).cpu().numpy()
ten_Ds = torch.cat(ten_Ds, dim=0).squeeze().cpu().numpy()  # (n, 10)
features = torch.cat(features, dim=0).squeeze().cpu().numpy()  # (n, 2048)
labels = torch.cat(labels, dim=0).squeeze().cpu().numpy()  # n
max_values = torch.cat(max_values, dim=0).squeeze().cpu().numpy()  # n
# ids = torch.cat(ids, dim=0).cpu().numpy() # n


In [10]:
# 保存numpy数组到文件
import matplotlib.pyplot as plt
from openTSNE import TSNE
import pandas as pd
# np.save('./temp/pic/AE_conf+_/labels.npy', labels)
# np.save('./temp/pic/AE_conf+_/max_values.npy', max_values)
# np.save('./temp/pic/AE_conf+_/ten_feature.npy', ten_Ds)
# np.save('./temp/pic/AE_conf+_/picture.npy', picture)
# np.save('./temp/pic/AE_conf+_/features.npy', features)

print(features.shape)
print(features[0])
print(ten_Ds.shape)
print(ten_Ds[0])


(50000, 43)
[ 2.7215562  -1.6361705  -1.7730188   6.5197697  -2.6341603   3.5241795
  0.29564893  0.1613927   2.123111   -0.87722903 -1.8821998  -1.7636718
 -2.662229   -1.671491   -0.26688498 -1.437731   -0.33685526 -1.5680554
 -2.7254262  -2.5143743  -2.2004719  -2.006973   -2.3314602  -0.9256389
 -2.4634888  -1.5855974  -2.2062507  -3.619819    0.11257693  0.08280019
 -0.6412584  -2.927784   -0.3935014  -0.8714204  -3.2288659  -1.4881755
 -1.9513947  -2.18876    -2.0772343  -2.1968708  -3.4484189  -2.4181497
 -2.095231  ]
(50000, 43)
[ 2.7215562  -1.6361705  -1.7730188   6.5197697  -2.6341603   3.5241795
  0.29564893  0.1613927   2.123111   -0.87722903 -1.8821998  -1.7636718
 -2.662229   -1.671491   -0.26688498 -1.437731   -0.33685526 -1.5680554
 -2.7254262  -2.5143743  -2.2004719  -2.006973   -2.3314602  -0.9256389
 -2.4634888  -1.5855974  -2.2062507  -3.619819    0.11257693  0.08280019
 -0.6412584  -2.927784   -0.3935014  -0.8714204  -3.2288659  -1.4881755
 -1.9513947  -2.18876   

In [ ]:
embed = TSNE(n_jobs=4).fit(features)  # N, 2
pd_embed = pd.DataFrame(embed)
print(pd_embed.shape)

# np.save('./temp/pic/AE_conf+_/embed_2dim.npy', pd_embed)
x = pd_embed.iloc[:, 0]
y = pd_embed.iloc[:, 1]
# loaded_labels = np.load('./temp/pic/AE/labels.npy')

# 绘制二维散点图
plt.figure(figsize=(8, 6))
# plt.scatter(x, y, c=loaded_labels, cmap='viridis')
plt.scatter(x, y, c=labels, cmap='viridis')
plt.colorbar(label='Labels')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Scatter Plot of Embeddings')
plt.grid(True)
plt.show()

In [11]:
from scipy import spatial

print(pd_embed[:5])

# 找到每个维度的最小值和最大值
min_x, min_y = np.min(pd_embed, axis=0)
max_x, max_y = np.max(pd_embed, axis=0)

# 缩放坐标到0到1的范围内
scaled_value = (pd_embed - [min_x, min_y]) / [max_x - min_x, max_y - min_y]

print("Scaled Coordinates:\n", scaled_value[:5])

tree = spatial.KDTree(data=scaled_value)

# torch.save(tree, "./static/data/CIFAR10/2D_kdTree/2D_kdTree_50000_png_2024-3-25.pt")
torch.save(tree, "GTSRB_cDCGAN_results/2D_kdTree_50000_png_features.pt")

           0          1
0 -14.334624 -14.059534
1 -19.629860  56.927075
2 -65.175466 -60.814551
3 -23.933736 -20.852627
4  26.891039  -9.420960
Scaled Coordinates:
           0         1
0  0.434617  0.429415
1  0.409387  0.767280
2  0.192382  0.206881
3  0.388881  0.397083
4  0.631039  0.451492
